![1](images/DIPOLE_DESSERT_400_1.png)
    
![2](images/DIPOLE_DESSERT_400_2.png)

![3](images/DIPOLE_DESSERT_400_3.png)
<!--     
![4](images/DIPOLE_DESSERT_400_4.png) -->

In [1]:
import json
import pennylane as qml
import pennylane.numpy as np
# Write any helper functions you need here

dev = qml.device('default.qubit', wires=[0,1,2])

@qml.qnode(dev)
def cloning_machine(coefficients, wire):
    
    """
    Returns the reduced density matrix on a wire for the cloning machine circuit.
    
    Args:
        - coefficients (np.array(float)): an array [c0,c1] containing the coefficients parametrizing
        the input state fed into the middle and bottom wires of the cloning machine.
        wire (int): The wire on which we calculate the reduced density matrix.

    Returns:
        - np.tensor(complex): The reduced density matrix on wire = wire, as returned by qml.density_matrix.
    
    """
    

    # Put your code here
    c0,c1 = coefficients
    qml.RY(2*np.arccos(c0/np.sqrt(2)+c1/np.sqrt(2)),[2])
    qml.RY(np.arccos(c1/np.sqrt(2-c0**2 - c1**2 -2*c0*c1)),[1])
    qml.CNOT([2,1])
    qml.RY(-np.arccos(c1/np.sqrt(2-c0**2 - c1**2 -2*c0*c1)),[1])
    qml.CNOT([2,1])

    qml.CNOT([0,1])
    qml.CNOT([0,2])
    qml.CNOT([1,0])
    qml.CNOT([2,0])
    # Return the reduced density matrix
    return qml.density_matrix([wire])

def fidelity(coefficients):
    
    """
    Calculates the fidelities between the reduced density matrices in wires 0 and 1 and the input state |0>.
    
    Args:
        - coefficients (np.array(float)): an array [c0,c1] containing the coefficients parametrizing
        the input state fed into the middle and bottom wires of the cloning machine.
    Returns:
        - (np.array(float)): An array whose elements are:
            - 0th element:  The fidelity between the output reduced state on wire 0 and the state |0>.
            - 1st element:  The fidelity between the output reduced state on wire 1 and the state |0>.    
    """
    

    # Put your code here
    d_0 = cloning_machine(coefficients, 0)
    d_1 = cloning_machine(coefficients, 1)
    f_0 = np.real(np.dot(np.array([1,0]),np.dot(d_0,np.array([1,0]))))
    f_1 = np.real(np.dot(np.array([1,0]),np.dot(d_1,np.array([1,0]))))
    return np.array([f_0,f_1])
# These functions are responsible for testing the solution.


def run(test_case_input: str) -> str:
    ins = json.loads(test_case_input)
    outs = fidelity(ins).tolist()
    
    return str(outs)


def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    u = cloning_machine([1/np.sqrt(3),1/np.sqrt(3)],1)
    for op in cloning_machine.tape.operations:
        assert (isinstance(op, qml.RX) or isinstance(op, qml.RY) or isinstance(op, qml.CNOT)), "You are using forbidden gates!"
    assert np.allclose(solution_output,expected_output, atol = 1e-4), "Not the correct fidelities"
